# Long-term Cognitive Networks for pattern classification

Long-term Cognitive Networks are trained with an inverse learning rule. In this model, the weights connecting the input neurons are coefficients of multiple regressions models while the weights connecting the temporal states with and outputs are computed using a learning method (the Moore–Penrose inverse method when no regularization is needed or the Ridge regression method when the model might overfit the data).

This notebook reproduces the results of the experiments reported in the original paper.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import pandas as pd

import random
import os

from ltcnclassifier.LTCN import LTCN

In [ ]:
def reset_random_seeds():
    os.environ['PYTHONHASHSEED']=str(42)
    np.random.seed(42)
    random.seed(42)

def kappa_scorer(Y_pred, Y):
    return cohen_kappa_score(np.argmax(Y, axis=1), np.argmax(Y_pred, axis=1))

def run_model(data, labels, n_classes, n_folds):
    
    reset_random_seeds()

    X = data[:,:-n_classes]
    Y = data[:,-n_classes:]
    
    train_errors = []
    test_errors = []
          
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    skf.get_n_splits(X, labels)
    
    for train_index, test_index in skf.split(X, labels):
        
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
 
        model = LTCN(method='inverse')

        # hyper-parameter tuning using grid search happens here!!
        param_grid = {'function': ['sigmoid', 'tanh'], 'phi': np.arange(0.5, 1.0, 0.1),
                      'T': [5, 10, 15]} #  'alpha': [0, 1.0E-2, 1.0E+2]

        kappa = make_scorer(kappa_scorer)
        grid_clf = GridSearchCV(model, param_grid, scoring=kappa, cv=n_folds, n_jobs=-1, error_score='raise')
        grid_clf.fit(X_train, Y_train)
        
        Y_pred_train = grid_clf.predict(X_train)
        train_errors.append(cohen_kappa_score(np.argmax(Y_train, axis=1), np.argmax(Y_pred_train, axis=1)))
        
        Y_pred_test = grid_clf.predict(X_test)
        test_errors.append(cohen_kappa_score(np.argmax(Y_test, axis=1), np.argmax(Y_pred_test, axis=1)))

    return sum(train_errors) / len(train_errors), sum(test_errors) / len(test_errors)

In [ ]:
files = os.listdir(os.pardir + '/datasets/')
print("running...")

n_folds = 5

# Input: A CSV file with normalized numerical values
# The pre-processing consists of the following steps:
#    1. We remove the classes whose number of instances is less than the number of folds
#    2. We one-hot encode the decision classes since they have no ordinal relationship

def preprocess(file):

    df = pd.read_csv(os.pardir + '/datasets/' + file, header=None)

    # select class values with at least n_folds ocurrences
    serie = df[df.columns[-1]].value_counts() >= n_folds 
    l = serie.index[serie.values].tolist()        
    df = df[df[df.columns[-1]].isin(l)]

    df_features = df.iloc[:, :-1] # here we have the data values
    df_classes = df.iloc[:, -1:] # here we have the classes

    # we need to encode the decision classes
    encoder = OneHotEncoder(handle_unknown='ignore').fit(df_classes.values)

    # use one-hot encode when no ordinal relationship exists
    data_encoded = encoder.transform(df_classes.values).toarray()

    # we have to merge the numerical variables with the encoded ones
    # we put axis=1 to specify that the append will be by column
    data = np.append(df_features.values, data_encoded, axis=1)

    # we return the one-hot encoded dataset, the original classes for stratification
    # in addition, we return the number of features and the number of unique classes

    return data, df_classes.values, len(df_features.columns), len(np.unique(df_classes.values))

with open('output.csv', 'a') as results_file:
    
    for file in files:
        
        if (not file.endswith('.csv')):
            continue
        
        data, labels, n_features, n_classes = preprocess(file)
        
        train_kappa, test_kappa = run_model(data, labels, n_classes, n_folds)
        results_file.write(file.replace('.csv', '') + "," + str(np.round(train_kappa, 4)) +
                           "," + str(np.round(test_kappa, 4)) + "\n")

        results_file.flush()
        print(file + ',' + str(np.round(train_kappa, 4)) + ',' + str(np.round(test_kappa, 4)))